In [1]:
import time
import tensorflow as tf
import cv2 as cv
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

2023-06-09 14:26:05.148652: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 14:26:05.179039: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 14:26:05.179532: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 14:26:05.974799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!ls

In [ ]:
!python model_main_tf2.py --model_dir=output/ --pipeline_config_path=ssd_mobilenet_v2_ft/pipeline.config

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ssd_mobilenet_v2/pipeline.config --trained_checkpoint_dir src/output/ --output_directory src/ssd_mobilenet_v2_ft2

In [2]:
PATH_TO_MODEL_DIR = 'efficientnet512_ft/'
PATH_TO_LABELS = '/home/irizqy/ml_ws/bangkit-ws/data/label_map.pbtxt'

In [4]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 17.751084804534912 seconds


In [5]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [12]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import os


rand_im = np.random.choice(os.listdir('/home/irizqy/ml_ws/bangkit-ws/data/temp'), 5)
# rand_im = '/home/irizqy/ml_ws/bangkit-ws/data/test/2_20230507_092258.jpg'

BASED_PATH = '/home/irizqy/ml_ws/bangkit-ws/data/temp'
IMAGE_PATHS = [os.path.join(BASED_PATH, im) for im in rand_im]
IMAGE_PATHS

warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
count = 0


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)
    
    if image_np.shape[2] == 4:
        image_np = image_np[:, :, :3]

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    # print(detections)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.5,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    im = Image.fromarray(image_np_with_detections)
    im.save(f"detected-logo2{count}.jpeg")
    count+=1
    print('Done')
    plt.show()

# sphinx_gallery_thumbnail_number = 2

Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/ktt_2.jpg... defaultdict(<class 'str'>, {(0.05700751394033432, 0.4025473892688751, 0.2991097867488861, 0.7482930421829224): 'Chartreuse'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/rfc9.jpg... defaultdict(<class 'str'>, {(0.09185022860765457, 0.23717626929283142, 0.2339421808719635, 0.873680591583252): 'Chartreuse'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/mcd_1.jpg... defaultdict(<class 'str'>, {(0.09174272418022156, 0.3433310389518738, 0.4638764560222626, 0.6533715724945068): 'Chartreuse', (0.487225741147995, 0.3388737142086029, 0.57807856798172, 0.7013683319091797): 'Chartreuse'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/sabana10.jpg... defaultdict(<class 'str'>, {(0.6756242513656616, 0.1792398989200592, 0.8114150762557983, 0.3966568410396576): 'Chartreuse'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/sabana10.jpg... defa

raw_detection_boxes <br>
detection_scores<br>
detection_boxes<br>
detection_anchor_indices<br>
raw_detection_scores<br>
detection_classes<br>
detection_multiclass_scores<br>
num_detections

In [ ]:
width, height, _ = image_np.shape
dets_index = np.where(detections['detection_scores'] >= .5)[0]
dets_index
for i in dets_index:
    ymin, xmin, ymax, xmax = detections['detection_boxes'][i]
    (left, right, top, bottom) = (xmin*width, xmax*width, ymin*height, ymax*height)
    # print(int(left), int(right), int(top), int(bottom))
    # im = cv.rectangle(image_np, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 3)
    # Image.fromarray(im).save('test.jpg')
    cropped_im = image_np[int(top):int(bottom), int(left):int(right)]
    Image.fromarray(cropped_im).save(f'cropped-logo{i}.jpg')